# Graphing issues

In [77]:
:dep plotters = { features = ["evcxr", "datetime"] }
:dep chrono = "*"
:dep serde = { features = ["derive"] }
:dep serde_json = "*"
:dep csv = "*"
:dep reqwest = "*"
:dep typed-headers = "*"
:dep bimap = "0.6.0"
:dep git2 = "*"

use std::error::*;
use std::fmt;
use std::collections::HashMap;
use std::str::FromStr;
use std::fs::*;
use std::path::{Path, PathBuf};
use chrono::{Datelike, Date, DateTime, NaiveDate, NaiveDateTime, TimeZone, offset::Utc};
use serde::*;
use plotters::style::text_anchor::{Pos, HPos, VPos};
use plotters::prelude::*;
use bimap::BiHashMap;

const MIN_YEAR: usize = 2015;
const MAX_YEAR: usize = 2021;

## Bugs

In [92]:
struct Row {
    date: Date<Utc>,
}

impl From<csv::StringRecord> for Row {
    fn from(record: csv::StringRecord) -> Self {
        let date = record.get(5).expect("no date");
        let date = NaiveDate::parse_from_str(date, "%m/%d/%Y").expect("bad date");
        let date = Date::from_utc(date, Utc);        
        
        Row { date }
    }
}

let data = csv::ReaderBuilder::new().delimiter(b'\t').has_headers(true).trim(csv::Trim::All).from_path("reviewed-bugs.tsv")?.into_records();
let data: Vec<Row> = data.filter_map(|r| r.ok()).map(Row::from).collect();

let min_date = data.iter().map(|r| r.date).min().expect("no min date");
let max_date = data.iter().map(|r| r.date).max().expect("no max date");

// count bugs per months
let year_month_count: HashMap<_, usize> = data.iter().fold(HashMap::<Date<Utc>, usize>::new(), |mut h, r| {
        let month = Utc.ymd(r.date.year(), r.date.month(), 1);
        h.entry(month).and_modify(|v| *v += 1).or_insert(1);
    
    h
});

let max_count: usize = year_month_count.values().max().cloned().unwrap_or(0);

let mut count = vec![];

// fill in any gaps in months
for year in MIN_YEAR..=MAX_YEAR {
    for month in 1..=12 {
        let month = Utc.ymd(year as i32, month as u32, 1);
        let mc = year_month_count.get(&month).cloned().unwrap_or(0);
        
        count.push((month, mc));
    }
}


let figure = evcxr_figure((1024, 768), move |root| {
    root.fill(&WHITE.mix(0.1));
    let mut chart = ChartBuilder::on(&root)
        .caption("Bugs by Month", ("Arial", 50).into_font().color(&WHITE))
        .margin(10)
        .x_label_area_size(50)
        .y_label_area_size(60)
        .build_cartesian_2d((min_date..max_date).monthly(), 0..(max_count+1))?;
    //        .build_cartesian_2d(0..count.len(), 0..max_count)?;

    chart.configure_mesh()
        //.disable_x_mesh()
        .bold_line_style(&WHITE.mix(0.3))
        //.light_line_style(&WHITE.mix(0.2))
        .y_desc("Count")
        .y_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Center)))    
        .x_desc("Month")
        .x_labels(50)
        .x_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Bottom)))
        .axis_desc_style(("sans-serif", 15).into_font().color(&WHITE))
        .axis_style(WHITE.mix(0.6).stroke_width(1))
        .draw()?;

    
     chart.draw_series(LineSeries::new(
         count.iter().cloned(),
         &RED,
     )).unwrap()
         .label("bugs")
         .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    
    

    chart.configure_series_labels()
        .background_style(&BLACK.mix(0.8))
        .label_font(("sans-serif", 15).into_font().color(&WHITE))
        .border_style(&BLUE)
        .draw()?;
    Ok(())
});

figure

Bugs by Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Count
 
 
Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
1
 
 
 
2
 
 
 
3
 
 
 
4
 
 
 
5
 
 
 
6
 
 
 
7
 
 
 
8
 
 
 
 
2015-12
 
 
 
2016-3
 
 
 
2016-6
 
 
 
2016-9
 
 
 
2016-12
 
 
 
2017-3
 
 
 
2017-6
 
 
 
2017-9
 
 
 
2017-12
 
 
 
2018-3
 
 
 
2018-6
 
 
 
2018-9
 
 
 
2018-12
 
 
 
2019-3
 
 
 
2019-6
 
 
 
2019-9
 
 
 
2019-12
 
 
 
2020-3
 
 
 
2020-6
 
 
 
2020-9
 
 
 
2020-12
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="70,707 70,707 70,707 70,707 70,707 70,707 70,707 70,707 70,707 70,707 70,635 80,707 95,707 111,707 125,707 141,635 156,635 171,635 186,707 202,420 217,635 232,563 248,707 263,707 278,635 294,707 308,563 323,563 338,635 354,563 369,707 384,707 400,348 415,133 430,707 445,563 461,707 476,635 490,563 506,492 521,492 536,635 551,563 567,563 582,635 597,707 613,420 628,563 643,635 659,492 673,635 688,707 703,635 719,635 734,635 749,707 765,635 780,707 795,420 810,707 826,635 841,707 856,635 871,420 886,707 902,563 917,707 932,635 948,635 963,635 978,563 993,635 1009,635 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 "/>
 
 
 
bugs

## Issues

In [19]:

use request::blocking::*;
use typed_headers::*;
use std::str:FromStr;

let content_type = mime::Mime::from_str("application/vnd.github.VERSION.text+json")?;

let client = Client::new();

let issues = client.get().header(ContentType.name(), content_type);

Error: error: failed to parse manifest at `/private/var/folders/qx/fvkvfhvx3mddcslgkycr4xp80000gn/T/.tmpgyCQIF/Cargo.toml`

Caused by:
  could not parse input as TOML

Caused by:
  unexpected character found: `;` at line 11 column 18

## Downloads from Crates.io

- Downloaded file from: https://static.crates.io/db-dump.tar.gz
- This is mentioned here: https://crates.io/data-access

In [3]:
// GET CRATE IDs

use std::collections::HashSet;

let db_data_path = Path::new("/Users/benjaminfry/Downloads/2021-02-15-crates-io-db-dump/data");
let trust_dns_crates = vec![
    "trust-dns", 
    "trust-dns-client",
    "trust-dns-server",
    "trust-dns-resolver", 
    "trust-dns-proto",
    "trust-dns-https",
    "trust-dns-rustls",
    "trust-dns-util",
    "trust-dns-openssl"].into_iter().collect::<HashSet<&'static str>>();

type CrateIds = BiHashMap<&'static str, u32>;

// crate info
fn parse_crates(db_path: &Path, trust_dns_crates: &HashSet<&'static str>) -> Result<CrateIds, Box<dyn Error>> {
    let file = File::open(db_path.join("crates.csv"))?;
    let mut csv = csv::ReaderBuilder::new().has_headers(true).flexible(false).from_reader(file);
    let mut out: CrateIds = CrateIds::with_capacity(trust_dns_crates.len());
    for r in csv.records() {
        let r = r?;
        let id: u32 = r.get(5).and_then(|s| s.parse().ok()).ok_or("bad record")?;
        let name = r.get(7).ok_or("bad record")?;
        
        if let Some(name) = trust_dns_crates.get(name) {
           out.insert(name, id);
        }
    }
    Ok(out)
}


let trust_dns_crate_ids = parse_crates(&db_data_path, &trust_dns_crates)?;
trust_dns_crate_ids

{"trust-dns-util" <> 33049, "trust-dns-openssl" <> 34233, "trust-dns-rustls" <> 13823, "trust-dns-https" <> 75534, "trust-dns-proto" <> 34235, "trust-dns-client" <> 176018, "trust-dns" <> 3001, "trust-dns-server" <> 7653, "trust-dns-resolver" <> 21069}

In [93]:
// GET CRATE VERSIONS
use de::Visitor;

struct DeDate;
impl<'de> Visitor<'de> for DeDate {
    type Value = DateTime<Utc>;

    fn expecting(&self, formatter: &mut fmt::Formatter) -> fmt::Result {
        write!(formatter, "expected date in format like '2016-02-13 16:55:27.298663'")
    }
    
    fn visit_str<E>(self, s: &str) -> Result<Self::Value, E>
    where
        E: de::Error,
    {
        Utc.datetime_from_str(s, "%Y-%m-%d %H:%M:%S%.f").map_err(|e| de::Error::custom(e))
    }
}

fn de_created_at<'de, D>(de: D) -> Result<DateTime<Utc>, D::Error> where D: Deserializer<'de> {
    de.deserialize_str(DeDate)
}

#[derive(Deserialize, Debug)]
struct CrateVersionRow {
    crate_id: u32,
    crate_size: Option<u64>,
    #[serde(deserialize_with = "de_created_at")]
    created_at: DateTime<Utc>,
    downloads: usize,
    features: String, // json
    id: u32,
    license: String,
    num: String, // ver
    published_by: Option<u32>,
    updated_at: String,
    yanked: char,
}

type VersionsMap = HashMap<u32, Vec<CrateVersionRow>>;

fn parse_versions(db_path: &Path, crate_ids: &CrateIds) -> Result<VersionsMap, Box<dyn Error>> {
    let file = File::open(db_path.join("versions.csv"))?;
    let mut csv = csv::ReaderBuilder::new().has_headers(true).flexible(false).from_reader(file);
    let mut out = VersionsMap::with_capacity(crate_ids.len() * 10);
    for r in csv.records() {
        let r = r?;
        let row = r.deserialize::<CrateVersionRow>(None)?;
        
        if crate_ids.contains_right(&row.crate_id) {
            out.entry(row.crate_id).or_insert_with(Vec::new).push(row);
        }
    }
    Ok(out)
}

let versions = parse_versions(&db_data_path, &trust_dns_crate_ids)?;
let version_ids = versions.values().flatten().map(|r| r.id).collect::<HashSet<u32>>();

let releases_by_month_map = versions.values().flatten()
    .map(|r| Utc.ymd(r.created_at.year(), r.created_at.month(), 1))
    .fold(HashMap::<Date<Utc>, usize>::new(), |mut m, d| {
    
    m.entry(d).or_insert(1usize);
    m
});

let mut releases_by_month = vec![];

// fill in any gaps in months
for year in MIN_YEAR..=MAX_YEAR {
    for month in 1..=12 {
        let month = Utc.ymd(year as i32, month as u32, 1);
        let mc = releases_by_month_map.get(&month).cloned().unwrap_or(0);
        
        releases_by_month.push((month, mc));
    }
}

let releases_by_month = releases_by_month;


let figure = evcxr_figure((1024, 256), move |root| {
    root.fill(&WHITE.mix(0.1));
    let mut chart = ChartBuilder::on(&root)
        .caption("Releases by Month", ("Arial", 50).into_font().color(&WHITE))
        .margin(10)
        .x_label_area_size(50)
        .y_label_area_size(60)
        .build_cartesian_2d((min_date..max_date).monthly(), 0..1usize)?;
    //        .build_cartesian_2d(0..count.len(), 0..max_count)?;

    chart.configure_mesh()
        //.disable_x_mesh()
        .bold_line_style(&WHITE.mix(0.3))
        //.light_line_style(&WHITE.mix(0.2))
        .y_desc("Count")
        .y_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Center)))    
        .x_desc("Month")
        .x_labels(50)
        .x_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Bottom)))
        .axis_desc_style(("sans-serif", 15).into_font().color(&WHITE))
        .axis_style(WHITE.mix(0.6).stroke_width(1))
        .draw()?;

    
//      chart.draw_series(LineSeries::new(
//          releases_by_month.iter().cloned(),
//          &RED,
//      )).unwrap()
//          .label("releases")
//          .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    
    chart.draw_series(Histogram::vertical(&chart)
            .style(RED.mix(0.5).stroke_width(0).filled())
            .data(releases_by_month)
    )?;
    

//     chart.configure_series_labels()
//         .background_style(&BLACK.mix(0.8))
//         .label_font(("sans-serif", 15).into_font().color(&WHITE))
//         .border_style(&BLUE)
//         .draw()?;
    Ok(())
});

figure

Releases by Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Count
 
 
Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
 
2015-12
 
 
 
2016-3
 
 
 
2016-6
 
 
 
2016-9
 
 
 
2016-12
 
 
 
2017-3
 
 
 
2017-6
 
 
 
2017-9
 
 
 
2017-12
 
 
 
2018-3
 
 
 
2018-6
 
 
 
2018-9
 
 
 
2018-12
 
 
 
2019-3
 
 
 
2019-6
 
 
 
2019-9
 
 
 
2019-12
 
 
 
2020-3
 
 
 
2020-6
 
 
 
2020-9
 
 
 
2020-12

In [91]:
// DOWNLOADS BY MONTH

type DownLoadsByMonth = HashMap<Date<Utc>, usize>;

fn month_from_str(date: &str) -> Result<Date<Utc>, Box<dyn Error>> {
    let date = NaiveDate::parse_from_str(date, "%Y-%m-%d")?;
    Ok(Utc.ymd(date.year(), date.month(), 1))
}


fn parse_version_downloads(db_path: &Path, version_ids: &HashSet<u32>) -> Result<DownLoadsByMonth, Box<dyn Error>> {
    let file = File::open(db_path.join("version_downloads.csv"))?;
    let mut csv = csv::ReaderBuilder::new().has_headers(true).flexible(false).from_reader(file);
    let mut out = HashMap::<Date<Utc>, usize>::new();
    for r in csv.records() {
        let r = r?;
        let mut r = r.iter();
        let date = month_from_str(r.next().ok_or("no date")?)?;
        let downloads: usize = r.next().and_then(|s: &str| s.parse().ok()).ok_or("bad dl")?;
        let version_id = r.next().and_then(|s: &str| s.parse().ok()).ok_or("bad dl")?;
        
        if version_ids.contains(&version_id) {
            *out.entry(date).or_insert(0) += downloads;
        }
    }
    Ok(out)
}

let downloads_by_month_hashed = parse_version_downloads(&db_data_path, &version_ids)?;

let min_date = downloads_by_month_hashed.keys().min().cloned().expect("no min date");
let max_date = downloads_by_month_hashed.keys().max().cloned().expect("no max date");
let max_count: usize = downloads_by_month_hashed.values().max().cloned().unwrap_or(0);

let mut downloads_by_month = vec![];

// fill in any gaps in months
for year in MIN_YEAR..=MAX_YEAR {
    for month in 1..=12 {
        let month = Utc.ymd(year as i32, month as u32, 1);
        let mc = downloads_by_month_hashed.get(&month).cloned().unwrap_or(0);
        
        downloads_by_month.push((month, mc));
    }
}

let downloads_by_month = downloads_by_month;

let figure = evcxr_figure((1024, 768), move |root| {
    root.fill(&WHITE.mix(0.1));
    let mut chart = ChartBuilder::on(&root)
        .caption("Downloads by Month", ("Arial", 50).into_font().color(&WHITE))
        .margin(10)
        .x_label_area_size(50)
        .y_label_area_size(60)
        .build_cartesian_2d((min_date..max_date).monthly(), 0..(max_count+1))?;
    //        .build_cartesian_2d(0..count.len(), 0..max_count)?;

    chart.configure_mesh()
        //.disable_x_mesh()
        .bold_line_style(&WHITE.mix(0.3))
        //.light_line_style(&WHITE.mix(0.2))
        .y_desc("Count")
        .y_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Center)))    
        .x_desc("Month")
        .x_labels(50)
        .x_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Bottom)))
        .axis_desc_style(("sans-serif", 15).into_font().color(&WHITE))
        .axis_style(WHITE.mix(0.6).stroke_width(1))
        .draw()?;

    
     chart.draw_series(LineSeries::new(
         downloads_by_month.iter().cloned(),
         &RED,
     )).unwrap()
         .label("downloads")
         .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    
    

    chart.configure_series_labels()
        .background_style(&BLACK.mix(0.8))
        .label_font(("sans-serif", 15).into_font().color(&WHITE))
        .border_style(&BLUE)
        .draw()?;
    Ok(())
});

figure

Downloads by Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Count
 
 
Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
100000
 
 
 
200000
 
 
 
300000
 
 
 
400000
 
 
 
 
2015-9
 
 
 
2015-12
 
 
 
2016-3
 
 
 
2016-6
 
 
 
2016-9
 
 
 
2016-12
 
 
 
2017-3
 
 
 
2017-6
 
 
 
2017-9
 
 
 
2017-12
 
 
 
2018-3
 
 
 
2018-6
 
 
 
2018-9
 
 
 
2018-12
 
 
 
2019-3
 
 
 
2019-6
 
 
 
2019-9
 
 
 
2019-12
 
 
 
2020-3
 
 
 
2020-6
 
 
 
2020-9
 
 
 
2020-12
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="70,707 70,707 70,707 70,707 70,707 70,707 70,707 70,707 70,706 84,706 99,706 113,706 128,706 142,706 156,706 171,706 185,706 200,706 214,706 229,706 244,706 258,706 273,706 287,706 302,706 317,705 330,706 345,706 359,705 374,705 388,705 403,706 418,705 432,704 447,705 461,705 476,703 491,695 504,690 519,690 533,684 548,675 562,671 577,674 592,678 606,638 621,632 635,634 650,595 665,566 678,519 693,514 707,480 722,459 736,448 751,420 766,465 780,436 795,472 809,427 824,376 839,382 853,323 868,313 882,252 897,178 911,266 926,282 941,202 955,157 970,169 984,198 999,63 1014,352 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 1014,707 "/>
 
 
 
downloads

In [90]:
// GIT COMMIT TRACKING

let trust_dns_path = Path::new("/Users/benjaminfry/Development/rust/trust-dns");

let trust_dns_repo = git2::Repository::open(trust_dns_path)?;

assert!(git2::RepositoryState::Clean == trust_dns_repo.state());

trust_dns_repo.head();
let mut commits = trust_dns_repo.revwalk()?;
commits.push_head()?;


let mut monthly_changes_map = HashMap::<Date<Utc>, usize>::new();
for commit in commits {
    let commit = commit?;
    let commit = trust_dns_repo.find_commit(commit)?;
    
    let parent = if let Some(parent) = commit.parents().next() {
        parent
    } else {
        continue
    };
    let parent_tree = parent.tree()?;
    let tree = commit.tree()?;
    let diff = trust_dns_repo.diff_tree_to_tree(Some(&tree), Some(&parent_tree), None)?;
    let stats = diff.stats()?;
    
    let time = NaiveDateTime::from_timestamp(commit.time().seconds(), 0);
    let month = Utc.ymd(time.year(), time.month(), 1);
    
    let changes = stats.insertions() + stats.deletions();
    *monthly_changes_map.entry(month).or_insert(0) += changes;
}
let monthly_changes_map = monthly_changes_map;

let min_date = monthly_changes_map.keys().min().cloned().expect("no min date");
let max_date = monthly_changes_map.keys().max().cloned().expect("no max date");
let max_count: usize = monthly_changes_map.values().max().cloned().unwrap_or(0);

let mut changes_by_month = vec![];

// fill in the gaps
for year in MIN_YEAR..=MAX_YEAR {
    for month in 1..=12 {
        let month = Utc.ymd(year as i32, month as u32, 1);
        let mc = monthly_changes_map.get(&month).cloned().unwrap_or(0);
        
        changes_by_month.push((month, mc));
    }
}

let figure = evcxr_figure((1024, 768), move |root| {
    root.fill(&WHITE.mix(0.1));
    let mut chart = ChartBuilder::on(&root)
        .caption("Changes by Month", ("Arial", 50).into_font().color(&WHITE))
        .margin(10)
        .x_label_area_size(50)
        .y_label_area_size(60)
        .build_cartesian_2d((min_date..max_date).monthly(), 0..(max_count+1))?;
    //        .build_cartesian_2d(0..count.len(), 0..max_count)?;

    chart.configure_mesh()
        //.disable_x_mesh()
        .bold_line_style(&WHITE.mix(0.3))
        //.light_line_style(&WHITE.mix(0.2))
        .y_desc("Changes")
        .y_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Center)))    
        .x_desc("Month")
        .x_labels(50)
        .x_label_style(TextStyle::from(("Arial", 15).into_font())
                        .color(&WHITE)
                        .pos(Pos::new(HPos::Right, VPos::Bottom)))
        .axis_desc_style(("sans-serif", 15).into_font().color(&WHITE))
        .axis_style(WHITE.mix(0.6).stroke_width(1))
        .draw()?;

    
     chart.draw_series(LineSeries::new(
         changes_by_month.iter().cloned(),
         &RED,
     )).unwrap()
         .label("downloads")
         .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    
    

    chart.configure_series_labels()
        .background_style(&BLACK.mix(0.8))
        .label_font(("sans-serif", 15).into_font().color(&WHITE))
        .border_style(&BLUE)
        .draw()?;
    Ok(())
});

figure

Changes by Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Changes
 
 
Month
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
20000
 
 
 
40000
 
 
 
60000
 
 
 
80000
 
 
 
100000
 
 
 
120000
 
 
 
 
2015-8
 
 
 
2015-11
 
 
 
2016-2
 
 
 
2016-5
 
 
 
2016-8
 
 
 
2016-11
 
 
 
2017-2
 
 
 
2017-5
 
 
 
2017-8
 
 
 
2017-11
 
 
 
2018-2
 
 
 
2018-5
 
 
 
2018-8
 
 
 
2018-11
 
 
 
2019-2
 
 
 
2019-5
 
 
 
2019-8
 
 
 
2019-11
 
 
 
2020-2
 
 
 
2020-5
 
 
 
2020-8
 
 
 
2020-11
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="70,697 70,697 70,697 70,697 70,697 70,697 70,697 70,640 84,677 99,673 113,685 128,685 142,680 157,658 171,677 186,689 200,642 215,673 229,693 244,679 259,696 273,635 288,419 302,616 317,656 332,473 345,686 360,619 374,661 389,681 403,654 418,689 433,582 447,377 462,603 476,673 491,657 506,668 519,653 534,648 548,624 563,677 577,649 592,689 607,677 621,63 636,673 650,692 665,582 680,693 693,645 708,669 722,690 737,689 751,683 766,690 781,690 795,568 810,686 824,601 839,680 854,688 868,692 882,675 897,685 911,674 926,695 941,692 955,668 970,661 984,686 999,686 1014,696 1014,697 1014,697 1014,697 1014,697 1014,697 1014,697 1014,697 1014,697 1014,697 1014,697 1014,697 "/>
 
 
 
downloads